# Question 1

In [1]:
!unzip /content/Humor,Hist,Media,Food.zip

Archive:  /content/Humor,Hist,Media,Food.zip
   creating: Humor,Hist,Media,Food/
  inflating: Humor,Hist,Media,Food/apsnet.txt  
  inflating: Humor,Hist,Media,Food/basehead.txt  
  inflating: Humor,Hist,Media,Food/c0dez.txt  
  inflating: Humor,Hist,Media,Food/chickenheadbbs.txt  
  inflating: Humor,Hist,Media,Food/cybrtrsh.txt  
  inflating: Humor,Hist,Media,Food/dalive  
  inflating: Humor,Hist,Media,Food/deadlysins.txt  
  inflating: Humor,Hist,Media,Food/draxamus.txt  
  inflating: Humor,Hist,Media,Food/exidy.txt  
  inflating: Humor,Hist,Media,Food/exylic.txt  
  inflating: Humor,Hist,Media,Food/golnar.txt  
  inflating: Humor,Hist,Media,Food/hack7.txt  
  inflating: Humor,Hist,Media,Food/hackingcracking.txt  
  inflating: Humor,Hist,Media,Food/hackmorality.txt  
  inflating: Humor,Hist,Media,Food/happyhack.txt  
  inflating: Humor,Hist,Media,Food/hermsys.txt  
  inflating: Humor,Hist,Media,Food/heroic.txt  
  inflating: Humor,Hist,Media,Food/iremember  
  inflating: Humor,Hist,Me

In [2]:
import pickle
import nltk
import time
import string
import glob

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Unzipping corpora/omw-1.4.zip.


True

In [3]:
tokenizer = nltk.tokenize.RegexpTokenizer(r"\w+")
lemmatizer = nltk.wordnet.WordNetLemmatizer()
stemmer = nltk.stem.PorterStemmer()
stopwords = nltk.corpus.stopwords.words()

def preprocess(line):
  '''
  Preprocess the data
    - Lowercase
    - Remove Extra Spaces
    - Lemmatize using WordNetLemmatizer()
    - Tokenize using RegexpTokenizer()
    - Remove stopwords
  '''
  line = " ".join(line.lower().split()) #lowercase and remove extra spaces
  #line = line.apply(lambda x: nltk.word_tokenize(x)) #tokenize
  line = [word for word in line.split() if not word in stopwords] #remove stopwords
  line = [lemmatizer.lemmatize(word) for word in line] #lemmatize
  line = tokenizer.tokenize(" ".join(line)) #tokenize
  return nltk.FreqDist(line)

data_dict = {}

for n, filee in enumerate(glob.glob('Humor,Hist,Media,Food/*')):
  with open(filee, "r", encoding='windows-1254', errors='ignore') as fp:
    fp = fp.read()
    freq_dict = preprocess(fp)
  for i in freq_dict:
    if i not in data_dict:
      data_dict[i] = [n]
    else:
      data_dict[i].append(n)

#save the index
with open('unigram_index_test.pkl', 'wb') as fp:
    pickle.dump(data_dict, fp)


In [4]:
#load the index
with open('unigram_index_test.pkl', 'rb') as fp:
    data_dict = pickle.load(fp)

def preprocessQuery(line):
  '''
  Preprocess the query
  '''
  tokenizer = nltk.tokenize.RegexpTokenizer(r"\w+")
  lemmatizer = nltk.wordnet.WordNetLemmatizer()
  stemmer = nltk.stem.PorterStemmer()
  line = " ".join(line.lower().split()) #lowercase and remove extra spaces
  #line = line.apply(lambda x: nltk.word_tokenize(x)) #tokenize
  line = [word for word in line.split() if not word in nltk.corpus.stopwords.words()] #remove stopwords
  line = [lemmatizer.lemmatize(word) for word in line] #lemmatize
  line = tokenizer.tokenize(" ".join(line)) #tokenize
  return line

def do_or(pl_1, pl_2):
    '''
    Function to perform 'OR' operation
    '''
    p1 = 0
    p2 = 0
    pl_or = []
    num_docs = len(set(pl_1).union(set(pl_2)))
    num_comparisons = 0
    while p1 != len(pl_1) and p2 != len(pl_2):
        if pl_1[p1] == pl_2[p2]:
            pl_or.append(pl_1[p1])
            num_comparisons+=1
            p1+=1
            p2+=1
        elif pl_1[p1] < pl_2[p2]:
            pl_or.append(pl_1[p1])
            num_comparisons+=1
            p1+=1
        else:
            pl_or.append(pl_2[p2])
            num_comparisons+=1
            p2+=1
    while p1 < len(pl_1):
        num_comparisons+=1
        pl_or.append(pl_1[p1])
        p1+=1
    while p2 < len(pl_2):
        num_comparisons+=1
        pl_or.append(pl_2[p2])
        p2+=1
    return pl_or, num_docs, num_comparisons

def do_and(pl_1, pl_2):
    '''
    Function to perform 'AND' operation
    '''
    p1 = 0
    p2 = 0
    pl_and = []
    num_docs = len(set(pl_1).union(set(pl_2)))
    num_comparisons = 0
    while p1 != len(pl_1) and p2 != len(pl_2):
        if pl_1[p1] == pl_2[p2]:
            pl_and.append(pl_1[p1])
            num_comparisons+=1
            p1+=1
            p2+=1
        elif pl_1[p1] < pl_2[p2]:
            num_comparisons+=1
            p1+=1
        else:
            num_comparisons+=1
            p2+=1
    return pl_and, num_docs, num_comparisons

def do_and_not(pl_1, pl_2):
    '''
    Function to perform 'AND NOT' operation
    '''
    p1 = 0
    p2 = 0
    pl_not = []
    num_docs = len(set(pl_1).union(set(pl_2)))
    num_comparisons = 0
    while p1 != len(pl_1) and p2 != len(pl_2):
        if pl_1[p1] == pl_2[p2]:
            num_comparisons+=1
            p1+=1
            p2+=1
        elif pl_1[p1] < pl_2[p2]:
            num_comparisons+=1
            pl_not.append(pl_1[p1])
            p1+=1
        else:
            num_comparisons+=1
            p2+=1
    return pl_not, num_docs, num_comparisons

def do_or_not(pl_1, pl_2):
    '''
    Function to perform 'OR NOT' operation
    '''
    total_docs = 1133
    not_pl_2 = set(range(total_docs)).difference(pl_2)
    return do_or(pl_1, list(not_pl_2))



In [5]:
query = 'telephone,paved, roads'
operation = ['OR NOT', 'AND NOT']
query = preprocessQuery(query)

print(query)

['telephone', 'paved', 'road']


## Run Query

In [6]:
queryOuts = data_dict[query[0]]
numDocsSet = set(data_dict[query[0]])
comps = 0
for i in range(len(operation)):
    if operation[i] == 'OR':
        temp = do_or(queryOuts, data_dict[query[i+1]])
        queryOuts = temp[0]
        numDocsSet=numDocsSet.union(set(queryOuts))
        comps+= temp[2]
    elif operation[i] == 'OR NOT':
        temp = do_or_not(queryOuts, data_dict[query[i+1]])
        queryOuts = temp[0]
        numDocsSet=numDocsSet.union(set(queryOuts))
        comps+= temp[2]
    elif operation[i] == 'AND':
        temp = do_and(queryOuts, data_dict[query[i+1]])
        queryOuts = temp[0]
        numDocsSet=numDocsSet.union(set(queryOuts))
        comps+= temp[2]
    elif operation[i] == 'AND NOT':
        temp = do_and_not(queryOuts, data_dict[query[i+1]])
        queryOuts = temp[0]
        numDocsSet=numDocsSet.union(set(queryOuts))
        comps+= temp[2]

print('Number of documents matched:', len(queryOuts))
print([glob.glob('Humor,Hist,Media,Food/*')[x] for x in queryOuts])
print(len(numDocsSet))
print('No. of comparisons required:', comps)

Number of documents matched: 998
['Humor,Hist,Media,Food/murphys.txt', 'Humor,Hist,Media,Food/wkrp.epi', 'Humor,Hist,Media,Food/talebeat.hum', 'Humor,Hist,Media,Food/thievco.txt', 'Humor,Hist,Media,Food/lipkovits.txt', 'Humor,Hist,Media,Food/lion.txt', 'Humor,Hist,Media,Food/antibiot.txt', 'Humor,Hist,Media,Food/hate.hum', 'Humor,Hist,Media,Food/dark.suc', 'Humor,Hist,Media,Food/shorties.jok', 'Humor,Hist,Media,Food/blkbean.txt', 'Humor,Hist,Media,Food/dym', 'Humor,Hist,Media,Food/byfb.txt', 'Humor,Hist,Media,Food/curry.txt', 'Humor,Hist,Media,Food/gumbo.txt', 'Humor,Hist,Media,Food/soporifi.abs', 'Humor,Hist,Media,Food/resolutn.txt', 'Humor,Hist,Media,Food/focaccia.brd', 'Humor,Hist,Media,Food/feggmagi.txt', 'Humor,Hist,Media,Food/rinaldo.jok', 'Humor,Hist,Media,Food/chickens.txt', 'Humor,Hist,Media,Food/butwrong.hum', 'Humor,Hist,Media,Food/murph.jok', 'Humor,Hist,Media,Food/analogy.hum', 'Humor,Hist,Media,Food/arnold.txt', 'Humor,Hist,Media,Food/skippy.txt', 'Humor,Hist,Media,Food/r

# Question 2

In [7]:
def preprocess_positional(line):
  '''
  Preprocess the data into positional index
    - Lowercase
    - Remove Extra Spaces
    - Lemmatize using WordNetLemmatizer()
    - Tokenize using RegexpTokenizer()
    - Remove stopwords
  '''
  line = " ".join(line.lower().split()) #lowercase and remove extra spaces
  #line = line.apply(lambda x: nltk.word_tokenize(x)) #tokenize
  line = [word for word in line.split() if not word in stopwords] #remove stopwords
  line = [lemmatizer.lemmatize(word) for word in line] #lemmatize
  line = tokenizer.tokenize(" ".join(line)) #tokenize
  dataPrep = {}
  for term in nltk.FreqDist(line):
    indices = [n for n, i in enumerate(line) if i == term]
    dataPrep[term] = indices
  return dataPrep

In [8]:
pos_dict = {}
for n, filee in enumerate(glob.glob('Humor,Hist,Media,Food/*')):
  with open(filee, "r", encoding='windows-1254', errors='ignore') as fp:
    fp = fp.read()
    freq_dict = preprocess_positional(fp)
  for i in freq_dict:
    if i not in pos_dict:
      pos_dict[i] = [(n, freq_dict[i])]
    else:
      pos_dict[i].append((n, freq_dict[i]))

In [9]:
#save index
with open('positional_index_test.pkl', 'wb') as fp:
    pickle.dump(pos_dict, fp)

In [10]:
def find_consecutive(l1, l2):
    lst = []
    for i in l1:
        for j in l2:
            if j-i == 1:
                lst.append(j)
    return lst

def do_and_pos(pl_1, pl_2):
    '''
    Function to perform 'AND' operation
    '''
    p1 = 0
    p2 = 0
    pl_and = []
    num_comparisons = 0
    while p1 != len(pl_1) and p2 != len(pl_2):
        if pl_1[p1][0] == pl_2[p2][0]:
            cons_lst= find_consecutive(pl_1[p1][1], pl_2[p2][1])
            if len(cons_lst)>0:
                pl_and.append((pl_1[p1][0], cons_lst))
            num_comparisons+=1
            p1+=1
            p2+=1
        elif pl_1[p1] < pl_2[p2]:
            num_comparisons+=1
            p1+=1
        else:
            num_comparisons+=1
            p2+=1
    return pl_and, num_comparisons


In [11]:
query = 'real aphrodisiacs obviously'
query = preprocessQuery(query)
print(query)


['real', 'aphrodisiac', 'obviously']


## Run Query

In [12]:
queryOuts = pos_dict[query[0]]
numDocsSet = set([x[0] for x in pos_dict[query[0]]])
comps = 0
for i in range(len(query)-1):
     temp = do_and_pos(queryOuts, pos_dict[query[i+1]])
     queryOuts = temp[0]
     comps+=temp[1]
     numDocsSet = numDocsSet.union(set([x[0] for x in queryOuts]))

print('Number of documents matched:', len(queryOuts))
print([glob.glob('Humor,Hist,Media,Food/*')[x[0]] for x in queryOuts])
print(len(numDocsSet))
print('No. of comparisons required:', comps)

Number of documents matched: 1
['Humor,Hist,Media,Food/aphrodis.txt']
327
No. of comparisons required: 326


In [13]:
a = pos_dict['playing']
b = pos_dict['first']
for i in a:
    for j in b:
        if i[0] == j[0]:
            for x1 in i[1]:
                for x2 in j[1]:
                    if x2-x1 == 1:
                        print(i, j)

(189, [46, 163, 214, 407, 413, 448, 502, 508]) (189, [4, 98, 113, 141, 153, 157, 165, 172, 207, 215, 222, 239, 280, 310, 326, 351, 367, 382, 414, 465, 509, 536, 609, 688, 696, 715, 731])
(189, [46, 163, 214, 407, 413, 448, 502, 508]) (189, [4, 98, 113, 141, 153, 157, 165, 172, 207, 215, 222, 239, 280, 310, 326, 351, 367, 382, 414, 465, 509, 536, 609, 688, 696, 715, 731])
(189, [46, 163, 214, 407, 413, 448, 502, 508]) (189, [4, 98, 113, 141, 153, 157, 165, 172, 207, 215, 222, 239, 280, 310, 326, 351, 367, 382, 414, 465, 509, 536, 609, 688, 696, 715, 731])
(431, [46, 163, 214, 408, 414, 449, 503, 509]) (431, [4, 98, 113, 141, 153, 157, 165, 172, 207, 215, 222, 239, 281, 311, 327, 352, 368, 383, 415, 466, 510, 537, 610, 690, 698, 717, 733])
(431, [46, 163, 214, 408, 414, 449, 503, 509]) (431, [4, 98, 113, 141, 153, 157, 165, 172, 207, 215, 222, 239, 281, 311, 327, 352, 368, 383, 415, 466, 510, 537, 610, 690, 698, 717, 733])
(431, [46, 163, 214, 408, 414, 449, 503, 509]) (431, [4, 98, 113,